In [1]:
library("phenopath")
library("reticulate")
np <- import("numpy")

In [2]:
set.seed(42)

In [3]:
phenopath_defaults <- function(observations, covariates, ...) {
    # suppressWarnings because I am well aware that it hasn't converged
    return(suppressWarnings(phenopath(observations, covariates, model_mu=TRUE, 
                                      maxiter=50, thin=10, verbose=FALSE, ...)))
}

In [4]:
spearman <- function(x,y){return(cor(x,y,method='spearman'))}

In [5]:
size = 11
seed = 42
number_droplets = 100000
number_batches = 5
results_dirname = 'results'

base_filename = paste(results_dirname, '/', size, '_strains.seed_', 
                      seed, '.', format(number_droplets, scientific=FALSE), '_droplets.iteration_',
                      '#', '.npz', sep='')

In [6]:
get_results <- function(phenopath_results, true_times) {
    uncensored_results <- interaction_effects(phenopath_results)
    censored_results <- significant_interactions(phenopath_results) * uncensored_results
    pearson <- abs(cor(true_times, trajectory(phenopath_results))) # same level of support if pseudotimes are flipped
    spearman <- abs(spearman(true_times, trajectory(phenopath_results)))
    
    results <- list("uncensored_results" = uncensored_results, 
                   "censored_results" = censored_results,
                   "pearson" = pearson,
                   "spearman" = spearman)
    return(results)
}

In [7]:
get_results_filename <- function(base_dir, scaling, iteration_number) {
    iteration_filename = paste('iteration_', iteration_number, '.npz', sep='')
    results_dir = file.path(paste(base_dir, '/', scaling, sep=''))
    results_filename = file.path(paste(results_dir, '/', iteration_filename, sep=''))
    return(results_filename)
}

save_results <- function(results_filename, results) {
    np$savez_compressed(results_filename,
    uncensored_results = results$uncensored_results,
    censored_results = results$censored_results,
    pearson = results$pearson,
    spearman = results$spearman)
}

In [8]:
all_results_dir = 'phenopath_results'
if (!dir.exists(all_results_dir)) {dir.create(all_results_dir)}

log_counts_cov_results_dir = file.path(paste(all_results_dir, '/', 'log_count_covariates', sep=''))
if (!dir.exists(log_counts_cov_results_dir)) {dir.create(log_counts_cov_results_dir)}

for (subdirectory in list.files(path=all_results_dir, full.names=T)) {
    scaled_results_dir = file.path(paste(subdirectory, '/', 'scaled', sep=''))
    if (!dir.exists(scaled_results_dir)) {dir.create(scaled_results_dir)}
    
    unscaled_results_dir = file.path(paste(subdirectory, '/', 'unscaled', sep=''))
    if (!dir.exists(unscaled_results_dir)) {dir.create(unscaled_results_dir)}
}

loop through the iterations of stored results

In [9]:
# very bad naughty code that uses lots of global variables
# this is a self-contained notebook however and not a software library.
# I don't want to rewrite the function and the function calls to unnecessarily be
# much longer when scope rules automatically do the right thing.
lapply_input <- function(iteration_number){
    
    filename = gsub("#", iteration_number, base_filename)
    npzfile = np$load(filename)

    read_log_counts = npzfile[["read_log_counts"]]
    # b/c 11th species has 0% frequency for 10%
    stopifnot(sum(read_log_counts[,11])==0)
    read_log_counts <- read_log_counts[,1:10]
    
    merged_droplets_per_batch <- dim(read_log_counts)[1]/number_batches
    true_times = c()
    for (i in 1:number_batches) {true_times <- append(true_times, rep(i, merged_droplets_per_batch))}
    
    start_time <- proc.time()
    log_counts_scaled <- phenopath_defaults(read_log_counts, read_log_counts, scale_y=TRUE)
    results <- get_results(log_counts_scaled, true_times)
    save_results(get_results_filename(log_counts_cov_results_dir, 'scaled', iteration_number), results)
    run_time <- proc.time() - start_time; print(run_time)
    
    start_time <- proc.time()
    log_counts_unscaled <- phenopath_defaults(read_log_counts, read_log_counts, scale_y=FALSE)
    results <- get_results(log_counts_unscaled, true_times)
    save_results(get_results_filename(log_counts_cov_results_dir, 'unscaled', iteration_number), results)
    run_time <- proc.time() - start_time; print(run_time)
    
    # this is supposed to be an embarrassingly parallel for loop, so memory usage should not change with number of iterations
    # but system monitor shows memory usage continually increasing. Hadley Wickham seems to have said that calling `gc`
    # manually for garbage collection should never be necessary, but honestly at this point I don't trust R so...
    gc()
    
    # controversial stylistically but consistent with Python style and since most programming is done
    # in Python, from a practical perspective it's better for me to use stylistic conventions that also work in Python.
    return(NULL)
}

In [10]:
lapply(1:100, lapply_input)

   user  system elapsed 
366.113  20.127 376.680 
   user  system elapsed 
562.006  41.822 607.685 
   user  system elapsed 
601.658  43.619 659.042 
   user  system elapsed 
597.909  42.506 647.985 
   user  system elapsed 
595.638  43.705 649.762 
   user  system elapsed 
598.082  40.059 646.847 
   user  system elapsed 
607.737  42.763 661.172 
   user  system elapsed 
579.484  41.455 620.709 
   user  system elapsed 
587.777  37.213 625.353 
   user  system elapsed 
573.299  37.460 606.349 
   user  system elapsed 
576.738  38.956 610.330 
   user  system elapsed 
569.135  41.926 606.500 
   user  system elapsed 
576.855  38.251 610.571 
   user  system elapsed 
571.303  40.550 606.795 
   user  system elapsed 
588.693  36.884 620.499 
   user  system elapsed 
572.716  39.734 606.774 
   user  system elapsed 
593.388  41.353 635.884 
   user  system elapsed 
634.435  60.496 730.629 
   user  system elapsed 
654.283  71.705 761.009 
   user  system elapsed 
624.691  62.022 708.510 


[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

[[30]]
NULL

[[31]]
NULL

[[32]]
NULL

[[33]]
NULL

[[34]]
NULL

[[35]]
NULL

[[36]]
NULL

[[37]]
NULL

[[38]]
NULL

[[39]]
NULL

[[40]]
NULL

[[41]]
NULL

[[42]]
NULL

[[43]]
NULL

[[44]]
NULL

[[45]]
NULL

[[46]]
NULL

[[47]]
NULL

[[48]]
NULL

[[49]]
NULL

[[50]]
NULL

[[51]]
NULL

[[52]]
NULL

[[53]]
NULL

[[54]]
NULL

[[55]]
NULL

[[56]]
NULL

[[57]]
NULL

[[58]]
NULL

[[59]]
NULL

[[60]]
NULL

[[61]]
NULL

[[62]]
NULL

[[63]]
NULL

[[64]]
NULL

[[65]]
NULL

[[66]]
NULL

[[67]]
NULL

[[68]]
NULL

[[69]]
NULL

[[70]]
NULL

[[71]]
NULL

[[72]]
NULL

[[73]]
NULL

[[74]]
NULL

[[75]]
NULL

[[76]]
NULL

[[77]]
NULL

[[78]]
NULL

[[79]]
NULL

[[80]]
NULL

[[81]]
NULL

[[82]]
NULL

[[83]]
NULL

[[84]]
NULL

[[85]]
NULL

[[86]]
NULL

[[87]]
NULL

[[88]]
NULL

[[89]]
NULL

[[90]]
NULL

[[91]]
NULL

[[92]]
NULL

[[93]]
NULL

[[94]]
NULL

[[95]]
NULL

[[96]]
NULL

[[97]]
NULL

[[98]]
NULL

[[99]]
NULL

[[100]]
NULL